In [66]:
import boto3
import tempfile
import logging

In [57]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [58]:
logging.debug('hello')

In [37]:
# Let's use Amazon S3
s3 = boto3.resource('s3')

In [38]:
# Tuplex needs a bucket.
# => create one tuplex-<iam-user> per default.
# => this is where stuff gets stored.

# layout bucket like this:
# tuplex-<iam-user>/notebooks
# tuplex-<iam-user>/data
# tuplex-<iam-user>/scratch

# upload lambda function as
# tuplex-runner

# -> add versioning to tuplex-runner! => allow for auto upload?


# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

aws-deepracer-3f4fbafa-e09c-412c-8491-baeb4b0bffb7
bbsn00
bmwcpo
tuplex
tuplex-public
tuplex-test


In [46]:
def current_iam_user():
    iam = boto3.resource('iam')
    user = iam.CurrentUser()
    return user.user_name.lower()

def default_lambda_name():
    return 'tuplex-lambda-runner'

def default_lambda_role():
    return 'tuplex-lambda-role'

def current_region():
    session = boto3.session.Session()
    region = session.region_name
    return region

def setup_aws(iam_user=current_iam_user(),
              lambda_name=default_lambda_name(),
              lambda_role=default_lambda_role(),
              region=current_region()
              ):
    logging.info('Setting up AWS Lambda backend for IAM user {}'.format(iam_user))
    logging.info('Configuring backend in zone: {}'.format(region))
    
    # check if iam user is found?
    # --> skip for now, later properly authenticate using assume_role as described in
    # https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_use_switch-role-api.html
    
    # step 1: create Lambda role if not exists
    iam = boto3.resource('iam')
    response = i
    
   
    
setup_aws()

INFO:root:Setting up AWS Lambda backend for IAM user leonhard
INFO:root:Configuring backend in zone: us-east-1


In [50]:
iam = boto3.resource('iam')
iam_client = boto3.client('iam')

In [92]:
lambda_role=default_lambda_role()

region = current_region()
overwrite = True


def create_lambda_role(iam_client, lambda_role):
    
    # Roles required for AWS Lambdas
    trust_policy = '{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Principal":{"Service":"lambda.amazonaws.com"},"Action":"sts:AssumeRole"}]}'
    lambda_access_to_s3 = '{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Action":["s3:*MultipartUpload*","s3:Get*","s3:ListBucket","s3:Put*"],"Resource":"*"}]}'
    lambda_invoke_others = '{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Action":["lambda:InvokeFunction","lambda:InvokeAsync"],"Resource":"*"}]}'

    iam_client.create_role(RoleName=lambda_role,
                           AssumeRolePolicyDocument=trust_policy,
                           Description='Auto-created Role for Tuplex AWS Lambda runner')
    logging.info('Created Tuplex AWS Lambda runner role ({})'.format(lambda_role))
    
def remove_lambda_role(iam_client, lambda_role):
    
    # detach policies...
        
    # delete role...
    iam_client.delete_role(RoleName=lambda_role)
    
try:
    response = iam_client.get_role(RoleName=lambda_role)
    logging.info('Found Lambda role from {}'.format(response['Role']['CreateDate']))
    
     # throw dummy exception to force overwrite
    if overwrite:
        remove_lambda_role(iam_client, lambda_role)
        logging.info('Overwriting existing role {}'.format(lambda_role))
        create_lambda_role(iam_client, lambda_role)
    
except iam_client.exceptions.NoSuchEntityException as e:
    logging.info('Role {} was not found in {}, creating ...'.format(lambda_role, region))
    create_lambda_role(iam_client, lambda_role)

INFO:root:Found Lambda role from 2021-11-03 19:15:37+00:00
INFO:root:Overwriting existing role tuplex-lambda-role
INFO:root:Created Tuplex AWS Lambda runner role (tuplex-lambda-role)


EntityAlreadyExistsException: An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name tuplex-lambda-role already exists.

In [76]:
!cat /var/folders/l7/8zgzcszx7z5gk7kk92f6nc1c0000gn/T/tmp8qrc12_k

cat: /var/folders/l7/8zgzcszx7z5gk7kk92f6nc1c0000gn/T/tmp8qrc12_k: No such file or directory


In [65]:
help(iam_client.create_role)

Help on method create_role in module botocore.client:

create_role(*args, **kwargs) method of botocore.client.IAM instance
    Creates a new role for your AWS account. For more information about roles, see `IAM roles <https://docs.aws.amazon.com/IAM/latest/UserGuide/WorkingWithRoles.html>`__ . For information about quotas for role names and the number of roles you can create, see `IAM and STS quotas <https://docs.aws.amazon.com/IAM/latest/UserGuide/reference_iam-quotas.html>`__ in the *IAM User Guide* .
    
    
    
    See also: `AWS API Documentation <https://docs.aws.amazon.com/goto/WebAPI/iam-2010-05-08/CreateRole>`_
    
    
    **Request Syntax** 
    ::
    
      response = client.create_role(
          Path='string',
          RoleName='string',
          AssumeRolePolicyDocument='string',
          Description='string',
          MaxSessionDuration=123,
          PermissionsBoundary='string',
          Tags=[
              {
                  'Key': 'string',
             

In [10]:
iam_client = boto3.client('iam')

In [12]:
iam = boto3.resource('iam')
account_summary = iam.AccountSummary()

In [14]:
account_summary.load()

In [17]:
account_summary.get_available_subresources()

[]

In [18]:
account_summary.summary_map

{'GroupPolicySizeQuota': 5120,
 'InstanceProfilesQuota': 1000,
 'Policies': 3,
 'GroupsPerUserQuota': 10,
 'InstanceProfiles': 1,
 'AttachedPoliciesPerUserQuota': 10,
 'Users': 3,
 'PoliciesQuota': 1500,
 'Providers': 0,
 'AccountMFAEnabled': 0,
 'AccessKeysPerUserQuota': 2,
 'AssumeRolePolicySizeQuota': 2048,
 'PolicyVersionsInUseQuota': 10000,
 'GlobalEndpointTokenVersion': 1,
 'VersionsPerPolicyQuota': 5,
 'AttachedPoliciesPerGroupQuota': 10,
 'PolicySizeQuota': 6144,
 'Groups': 2,
 'AccountSigningCertificatesPresent': 0,
 'UsersQuota': 5000,
 'ServerCertificatesQuota': 20,
 'MFADevices': 0,
 'UserPolicySizeQuota': 2048,
 'PolicyVersionsInUse': 23,
 'ServerCertificates': 0,
 'Roles': 18,
 'RolesQuota': 1000,
 'SigningCertificatesPerUserQuota': 2,
 'MFADevicesInUse': 0,
 'RolePolicySizeQuota': 10240,
 'AttachedPoliciesPerRoleQuota': 10,
 'AccountAccessKeysPresent': 1,
 'GroupsQuota': 300}

In [20]:
user = iam.CurrentUser()

In [22]:
user.user_name

'Leonhard'

In [23]:
user.user_id

'AIDAIJ6K567DOELIXHE52'

In [25]:
user.get_available_subresources()

[]